The following code corresponds to the paper: *“A soundscape approach for a short-term acoustic monitoring of a Critically Endangered Cuban frog”*
It comprises the elaboration of Figure 2 (An Oscillogram, Spectrogram and Power Spectro of a typical call from _E. bartonsmithi_) and Figure 7 A double y axis plot to see the variation of acoustical index and call rate from _E. bartonsmithi_ in time. There will be a Cell for each Figure

In [ ]:
#Figure 2 

from matplotlib.gridspec import GridSpec
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import spectrogram
from scipy.signal import periodogram

# ======= GET and transform the DATA =======

# Read the wav file
sample_rate, data = wavfile.read('Barton_sample.wav')

# Select the time slice from 20 to 40 seconds
start_time = 20
end_time = 40
start_sample = start_time * sample_rate
end_sample = end_time * sample_rate
data = data[start_sample : end_sample]

# Define the start and end times for the zoomed oscillogram
zoom_start_time = 9.9
zoom_end_time = 10.50
zoom_start_sample = int(zoom_start_time * sample_rate)
zoom_end_sample = int(zoom_end_time * sample_rate)

# Extract the zoomed data and generate the corresponding time array
zoom_data = data[zoom_start_sample : zoom_end_sample]
zoom_time = np.arange(zoom_start_sample, zoom_end_sample) / sample_rate

# Generate the time axis for oscillogram
time = np.arange(0, len(data)) / sample_rate
# Generate the spectrogram
freqs, times, Sxx = spectrogram(data, fs=sample_rate, nperseg=4096, noverlap=2048)

# Select the frequency slice from 1000 to 6000 Hz
Sxx = Sxx[(freqs >= 900) & (freqs <= 6500), :]
freqs = freqs[(freqs >= 900) & (freqs <= 6500)]

# Generate the time array for the spectrogram
times = np.linspace(0, 20, Sxx.shape[1])

# Define the start and end times for the power spectrum
ps_start_time = 9.9
ps_end_time = 10.5
ps_start_sample = int(ps_start_time * sample_rate)
ps_end_sample = int(ps_end_time * sample_rate)

# Extract the data for the power spectrum and generate the corresponding power spectrum
ps_data = data[ps_start_sample : ps_end_sample]
f, Pxx_den = periodogram(ps_data, fs=sample_rate)

# ======= Generating the FIGURE =======

# Create a new figure with a custom grid of subplots
fig = plt.figure(figsize=(15, 10))
gs = GridSpec(2, 3, figure=fig)

# Generate the original oscillogram
ax1 = fig.add_subplot(gs[0, :2])
ax1.plot(time, data, color = "#3066be")
ax1.set_title('Original Oscillogram')
ax1.set_ylabel('Amplitude')

# Generate the zoomed oscillogram
ax2 = fig.add_subplot(gs[0, 2])
ax2.plot(zoom_time, zoom_data, color = "#3066be")
ax2.set_title('Zoomed Oscillogram (9-11s)')
ax2.set_ylabel('Amplitude')

# Generate the spectrogram
ax3 = fig.add_subplot(gs[1, :2])
cax = ax3.pcolormesh(times, freqs, 10 * np.log10(Sxx), shading='auto', cmap='inferno')
ax3.set_title('Spectrogram')
ax3.set_xlabel('Time (s)')
ax3.set_ylabel('Frequency (Hz)')

fig.colorbar(cax, ax=ax3, orientation='horizontal', pad=0.1, label='Intensity (dB)')

# Generate the power spectrum
f, Pxx_den = periodogram(ps_data, fs=sample_rate)

# Find the frequency with the highest amplitude
max_freq = f[np.argmax(Pxx_den)]

# Create a mask for the frequencies between 500 and 5000 Hz
mask = (f >= 500) & (f <= 5000)

# Apply the mask to the frequencies and power spectral density
f_masked = f[mask]
Pxx_den_masked = Pxx_den[mask]

# Generate the power spectrum plot
ax4 = fig.add_subplot(gs[1, 2])
ax4.semilogy(f_masked, Pxx_den_masked, color = "#06257a")
ax4.axvline(max_freq, color='k')  # Add a vertical line at the frequency with the highest amplitude
ax4.set_title('Power Spectrum (9.9-10.5s)')
ax4.set_xlabel('Frequency [Hz]')
ax4.set_ylabel('PSD [V**2/Hz]')
ax4.set_xlim(500, 5000)  # Set the x-axis limits to 500 and 5000 Hz

# Display the figure
plt.tight_layout()
plt.savefig('zoomed_spectronoscillo_power_spectrum.png', dpi=600)
plt.show()


In [ ]:
#Figure 7
import numpy as np
import pandas as pd
import datetime as dtime
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt


# get strided array for a given window length and offset
def stride(arr, win_len, offset=1):
    n_windows = (arr.shape[0] - win_len) // offset + 1
    idx = np.arange(win_len)[None,:] + offset * np.arange(n_windows)[:,None]
    return arr[idx]

# get right spine ready to be customized
def activate_spines(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    plt.setp(ax.spines.values(), visible=False)
    ax.spines['right'].set_visible(True)


# ======= GET DATA =======
df = pd.read_excel('double_y_axis_data.xlsx')
minutes = []
for time in df['time']:
    dt = dtime.datetime.strptime(str(time), '%Y-%m-%d %H:%M:%S')
    minutes.append(dt.hour * 60 + dt.minute)
minutes = np.array(minutes)
df['Minutes'] = minutes  # new column with time units in minutes


# ======= PLOTTING =======
fs = 15
color_indexs = '#333333'
color_crates = '#1f9e89'

index_names = df.columns[1:8]  # indexes to plot
for i, iname in enumerate(index_names):
    fig, ax = plt.subplots(figsize=(7,4))
    tax = ax.twinx()

    df_ = df[df['Season'] == 'October']  # just keep data recorded in October
    df_ = df_.reset_index(drop=True)
    minutes = df_['Minutes']

    isort = np.argsort(minutes)  # sort data for increasing time values
    time = minutes[isort] / 60
    index = df_[iname][isort]
    call_rate = df_['call_rate'][isort]

    ibreak = np.where(np.ediff1d(time) > .3)[0]  # split data to remove non recorded periods
    lim_break = [None, *(ibreak + 1), None]
    j = 1
    l1, l2 = lim_break[j], lim_break[j+1]
    t = time[l1:l2]
    t_mask = (5 <= t) & (t <= 9)
    ax.plot(t[t_mask], gaussian_filter1d(index[l1:l2], sigma=20)[t_mask], color=color_indexs)  # spline using Gaussian kernel
    
    ax.spines['left'].set_edgecolor(color_indexs)
    ax.tick_params(axis='y', labelsize=fs, colors=color_indexs)
    ax.tick_params(labelsize=fs)
    ax.set_xlabel('Hour of the day', fontsize=fs)
    ax.set_ylabel(iname.replace('_', ' '), fontsize=fs, color=color_indexs)
    
    cr = np.array(call_rate[l1:l2])  # discard null call rate values as well as np.nan
    mask = (cr != 0) & ~np.isnan(cr)

    win_len = 5
    st_crate = stride(cr[mask], win_len)  # stride call rate time series to use it as sliding window
    median_crate = np.nanmedian(st_crate, axis=1)  # compute median locally
    median_crate = gaussian_filter1d(median_crate, sigma=15)  # apply Gaussian filter
    t = time[l1:l2][mask][:-(win_len-1)]  # broadcast time data accordingly
    t_mask = (5 <= t) & (t <= 9)  # get desired interval
    tax.plot(t[t_mask], median_crate[t_mask], color=color_crates)

    activate_spines(tax)
    tax.spines['right'].set_edgecolor(color_crates)
    tax.tick_params(axis='y', labelsize=fs, colors=color_crates)
    tax.set_ylabel('Call rate (calls/min)', fontsize=fs, color=color_crates)

    # plt.savefig('%s.png' % iname, dpi=600, bbox_inches='tight')
plt.show()
